<a href="https://colab.research.google.com/github/krishnannarayanaswamy/astra-ai-demo/blob/main/202308_Performance_Fun.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing needed libraries, preping the env

In [ ]:
# Some preparation
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.2 MB/s eta 0:00:00


In [ ]:
# Ask user for OpenAI API Key
from getpass import getpass
OPENAIKEY = getpass('Please enter your OpenAI API Key: ')

Please enter your OpenAI API Key: ··········


# 1. Optimizing the Output - FSL and COT to reduce Hallucinations

In [ ]:
import openai
import time

# Set up OpenAI API key
openai.api_key = OPENAIKEY

#define initial prompt
prompt = """
John has 2 houses. Each house has 3 bedrooms and there are 2 windows in each bedroom.
Each house has 1 kitchen with 2 windows. Also, each house has 5 windows that are not in the bedrooms or kitchens.
How many windows are there in John's houses? Explain your reasoning.
"""

# Generate final response
response = openai.Completion.create(
    engine="text-davinci-003",
    prompt=prompt,
    max_tokens=100,
    n=1,
    stop=None,
    temperature=0.0,
)

print(f"Response: {response.choices[0].text.strip()}")



ModuleNotFoundError: ignored

**boah, not really true, is it?**

In [ ]:
# let's try with some samples, chain of thoughts (CoT)
# model: text-davinci-003
prompt = """
Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
A: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. The answer is 6.

Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
A: There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. The answer is 5.

Q: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
A: Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they
had 74 - 35 = 39. The answer is 39.

Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?
A: Jason started with 20 lollipops. Then he had 12 after giving some to Denny. So he gave Denny 20 - 12 = 8. The answer is 8.

Q: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?
A: Shawn started with 5 toys. If he got 2 toys each from his mom and dad, then that is 4 more toys. 5 + 4 = 9. The answer is 9.

Q: John has 2 houses. Each house has 3 bedrooms and there are 2 windows in each bedroom.
Each house has 1 kitchen with 2 windows. Also, each house has 5 windows that are not in the bedrooms or kitchens.
How many windows are there in John's houses?
A:
"""

# Generate final response
response = openai.Completion.create(
    engine="text-davinci-003",
    prompt=prompt,
    max_tokens=100,
    n=1,
    stop=None,
    temperature=0.9,
)

print(f"Response: {response.choices[0].text.strip()}")



Response: John has 2 houses. Each house has 3 bedrooms with 2 windows each. That makes 6 windows for each house. Each house also has 1 kitchen with 2 windows and 5 windows that are not in the bedrooms or kitchens. So for each house, there are 6 + 2 + 5 = 13 windows. Since there are 2 houses, there are 13 x 2 = 26 windows in total. The answer is 26.


# **Summary**

The chain-of-thought prompting clearly increases the ability of LLMs at certain tasks. I suggest reading the entire paper if you’d like to learn more about the experiments they conducted.

It is important to note that, as also mentioned in the paper, the benefits of chain-of-thought prompting only become evident when applied to models with approximately 100 billion parameters, and it doesn’t significantly enhance the performance of smaller models.

The experiment results yield to the conclusion that smaller models produce fluent but illogical chains of thought, which leads to a worse performance than standard prompting.

**References**

Wei, Jason, et al. (2022). Chain-of-Thought Prompting Elicits Reasoning in Large Language Models. ArXiv. https://arxiv.org/abs/2201.11903



---
# 2. Discussing relevance using distance calculations


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Define vector database
vector_database = [
    np.array([0.2, 0.3, 0.8]),
    np.array([0.5, 0.1, 0.9]),
    np.array([0.7, 0.5, 0.2])
]

# Define user query
user_query = np.array([0.3, 0.6, 0.2])

# Calculate cosine similarity and retrieve relevant results
results = []
for i, vector in enumerate(vector_database):
    similarity = cosine_similarity([user_query], [vector])[0][0]
    results.append((i, similarity))

# Sort results by relevance
results.sort(key=lambda x: x[1], reverse=True)

# Print top k results
k = 2
for i in range(min(k, len(results))):
    index, similarity = results[i]
    print(f"Result {i+1}: Index {index}, Similarity {similarity}")

Result 1: Index 2, Similarity 0.889646241113611
Result 2: Index 0, Similarity 0.651203294055074


In this code, we have a vector database represented by a list of NumPy arrays. We also have a user query represented by a NumPy array. We calculate the cosine similarity between the user query and each vector in the database using the cosine_similarity function from scikit-learn.

The results are stored in a list of tuples, where each tuple contains the index of the vector in the database and its similarity to the user query. We then sort the results based on similarity in descending order.
Finally, we print the top k results, where k is the desired number of relevant results to display.

This code demonstrates how cosine similarity can be used to measure the relevance of results in a vector database based on a user query. You can modify the vector database, user query, and the value of k to suit your specific needs.

**step-by-step explanation of how the cosine similarity is calculated:**

1. The code snippet uses the cosine_similarity function from the scikit-learn library. This function takes two arrays as input and calculates the cosine similarity between them.

2. The cosine similarity is calculated by taking the dot product of the two vectors and dividing it by the product of their magnitudes. Mathematically, it can be represented as:
Cosine Similarity Formula
Here, A and B represent the two vectors, and ||A|| and ||B|| represent their respective magnitudes.

3. The cosine similarity value ranges from -1 to 1. A value of 1 indicates that the vectors are identical, 0 indicates no similarity, and -1 indicates complete dissimilarity.
4. In the code snippet, the cosine similarity is calculated for each vector in the vector database with the user query vector.
The results are stored in a list of tuples, where each tuple contains the index of the vector in the database and its similarity to the user query.
5. The results are then sorted in descending order based on similarity, so that the most similar vectors appear first.

**There are several distance metrics used in vector search besides cosine similarity. Some of the commonly used distance metrics are:**

1. Euclidean distance: This is the straight-line distance between two points in a multidimensional space. It is calculated as the square root of the sum of the squared differences between the corresponding components of the two vectors.

2. Manhattan distance: This is the distance between two points measured along the axes at right angles. It is calculated as the sum of the absolute differences between the corresponding components of the two vectors.

3. Hamming distance: This is the number of positions at which the corresponding components of two vectors are different. It is commonly used in binary data, such as DNA sequences or error-correcting codes.

4. Mahalanobis distance: This is a measure of the distance between a point and a distribution. It takes into account the covariance structure of the data and is useful for data with correlated features.

5. Bray-Curtis dissimilarity: This is a measure of dissimilarity between two vectors that takes into account the relative abundance of the components. It is commonly used in ecology and biology.

These distance metrics can be used in vector search and recommendation systems to measure the similarity or dissimilarity between vectors. The choice of distance metric depends on the specific application and the nature of the data.

Citations:
[1] https://towardsdatascience.com/9-distance-measures-in-data-science-918109d069fa
[2] https://weaviate.io/blog/distance-metrics-in-vector-search

[3] https://medium.datadriveninvestor.com/cosine-similarity-cosine-distance-6571387f9bf8

[4] https://www.learndatasci.com/glossary/cosine-similarity/

[5] https://www.imaurer.com/which-vector-similarity-metric-should-i-use/

[6] https://www.machinelearningplus.com/nlp/cosine-similarity/

---
**How do we have to change the above code to demonstrate Recall@K?**

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Define vector database
vector_database = [
    np.array([0.2, 0.3, 0.8]),
    np.array([0.5, 0.1, 0.9]),
    np.array([0.7, 0.5, 0.2]),
    np.array([0.8, 0.4, 0.1])
]

# Define user query
user_query = np.array([0.3, 0.6, 0.2])

# Calculate cosine similarity and retrieve relevant results
results = []
for i, vector in enumerate(vector_database):
    similarity = cosine_similarity([user_query], [vector])[0][0]
    results.append((i, similarity))

# Sort results by relevance
results.sort(key=lambda x: x[1], reverse=True)

# Calculate Recall@k
k = 2
relevant_results = [result[0] for result in results[:k]] # hint: results[:k] retrieves elements from index 0 to index k-1.
recall = len(set(relevant_results)) / len(vector_database)

print(f"Recall@{k}: {recall}")

Recall@2: 0.5


The results are stored in a list of tuples, where each tuple contains the index of the vector in the database and its similarity to the user query. We then sort the results based on similarity in descending order.
We calculate Recall@k by considering the top k relevant results.

In this example, we set k to 2. We extract the indices of the top k relevant results and calculate the recall as the ratio of the number of relevant results to the total number of vectors in the database.
Finally, we print the Recall@k value.

**Question: What happens if we add another distinct (1) vector to our database?  What is the output then? What will be the output if you increase K to 3?**

---

**How do we have to change the above code to demonstrate AP (Average Precision)?**

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Define vector database
vector_database = [
    np.array([0.2, 0.3, 0.8]),
    np.array([0.5, 0.1, 0.9]),
    np.array([0.7, 0.5, 0.2])
]

# Define user query
user_query = np.array([0.3, 0.6, 0.2])

# Calculate cosine similarity and retrieve relevant results
results = []
for i, vector in enumerate(vector_database):
    similarity = cosine_similarity([user_query], [vector])[0][0]
    results.append((i, similarity))

# Sort results by relevance
results.sort(key=lambda x: x[1], reverse=True)

# Calculate Recall@k
k = 2
relevant_results = [result[0] for result in results[:k]]
recall = len(set(relevant_results)) / len(vector_database)

print(f"Recall@{k}: {recall}")

# Calculate Average Precision (AP)
precision = []
for i, result in enumerate(results):
    if result[0] in relevant_results:
        precision.append(len(set(relevant_results[:k]).intersection(set([r[0] for r in results[:i+1]]))) / (i+1))

ap = sum(precision) / len(relevant_results)

print(f"AP: {ap}")

Recall@2: 0.6666666666666666
AP: 1.0


In this updated code, after calculating Recall@k, we compute the precision values at each relevant result position and store them in the precision list.

We then calculate the AP by taking the average of the precision values.
Please note that the code assumes that the relevance of results is binary (relevant or not relevant).

If you have a different relevance scoring system, you may need to modify the code accordingly.